# V20: Ghost/scattered-light pixel features

Computes simple pixel-level artifact features from a TPF difference image to flag ghost/scattered-light-like behavior.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

# Use a representative sector TPF for pixel-based checks.
sector = 83
tpf = ds.tpf_by_sector[sector]

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended', tpf=tpf)


r = session.run('V20')

print(json.dumps({'status': r.status, 'flags': list(r.flags), 'metrics': r.metrics}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "aperture_contrast": 95.52915209791492,
    "aperture_pixels_used": 23,
    "background_trend": 0.023589383897246057,
    "edge_gradient_strength": 0.012801679978130275,
    "ghost_like_score": 0.25373548353074543,
    "in_aperture_depth": 10.81657077955163,
    "out_aperture_depth": 0.1132279575606923,
    "prf_likeness": 0.40448878741565075,
    "scattered_light_risk": 0.029442413621580548,
    "spatial_uniformity": 0.03882749624251436
  },
  "status": "ok"
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt

from tess_vetter.plotting import plot_ghost_features

step_id = '22_v20_ghost_features'
fname = 'V20_ghost_features.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    ax, _cbar = plot_ghost_features(r, ax=ax)
    ax.set_title('V20: Ghost/scattered-light pixel features')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/22_v20_ghost_features/V20_ghost_features.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/22_v20_ghost_features/V20_ghost_features.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/22_v20_ghost_features/V20_ghost_features.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** low scattered-light risk (~0.03) and a moderate ghost-like score (~0.25) for this sector’s TPF; the difference image is not strongly edge-dominated.
- **Why this is useful:** pixel-level artifacts often produce diffuse/edge-driven signals; a localized, aperture-consistent difference image is more compatible with an on-target astrophysical signal.
- **Interpretation:** this check does not strongly indicate scattered light/ghost contamination for the chosen sector, but it is only a partial screen (per-sector, per-aperture).
- **Next step:** V21 sector consistency to quantify whether transit depth is stable across sectors.

</details>
